In [202]:
from openpyxl import load_workbook

In [203]:
# Address Data

In [221]:
file_path_addresses = 'C:/Users/micha/Documents/01_high_prio_bus/00 dev/python/django/env_3.10_projects/scerp/billing/fixtures/Abonnenten Gebühren einzeilig.xlsx'

In [222]:
# Load the workbook and select a sheet
wb = load_workbook(file_path_addresses)
ws = wb.active  # Or wb['SheetName']

In [223]:
rows = [row for row in ws.iter_rows(values_only=True)]

In [224]:
addresses = {}

In [225]:
for row_nr, row in enumerate(rows):
    cells = row
    if cells[0] and (isinstance(cells[0], int) or isinstance(cells[0], float)):
        (
            abo_nr, r_empf, persnr, namevorname, _, strasse, plz_ort, _, 
            tarif, periode, tarifbez, basis, ansatznr, ansatz, tage, 
            betrag, inklmwst, steuercodezähler, berechnungscodezähler, 
            steuercodegebühren, berechnungscodegebühren, gebührentext,
             gebührenzusatztext
        ) = row
        address_data[abo_nr] = {
            'plz': plz_ort.split(' ')[0],
            'city': plz_ort.split(' ')[1],
            'address': strasse
        }

In [226]:
address_data

{585: {'plz': '5000', 'city': 'Aarau', 'address': 'Hombergstrasse 12'},
 406: {'plz': '8754',
  'city': 'Netstal',
  'address': 'Industrie Kleinzaun, Postfach 197'},
 44: {'plz': '4710', 'city': 'Balsthal', 'address': 'Tiergartenweg 1'},
 1: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Allmend 18'},
 54: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Mittelgäustr. 16'},
 37: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Höllweg 1'},
 38: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Höllweg 1'},
 56: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Mittelgäustr. 26'},
 342: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Schulstrasse 8'},
 51: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Mittelgäustr. 14'},
 207: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Mittelgäustr. 54'},
 208: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Mittelgäustr. 54'},
 328: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Feldstrasse 2'},
 53: {'plz': '4617', 'city': 'Gunzgen', 'address': 'Mittelgäustr

In [227]:
# Counter Data

In [228]:
file_path = 'C:/Users/micha/Documents/01_high_prio_bus/00 dev/python/django/env_3.10_projects/scerp/billing/fixtures/Abonnenten mit Zähler und Gebühren.xlsx'

In [229]:
# Load the workbook and select a sheet
wb = load_workbook(file_path)
ws = wb.active  # Or wb['SheetName']

In [230]:
rows = [row for row in ws.iter_rows(values_only=True)]

In [231]:
# Iterate row by row
addresses = []
building = {}
subscriber = {}
subscription = {}
counter = {}
montage = {}
product = {}
measurements = []

In [232]:
def get_company(name):
    if ' AG' in name:
        return name
    elif ' gmbh' in name.lower():
        return name        
    elif 'genossenschaft' in name.lower():
        return name    
    elif 'verband' in name.lower():
        return name
    elif 'verein' in name.lower():
        return name
    elif 'gemeinde' in name.lower():
        return name
    elif 'STWEG' in name.upper():
        return name
        
    return None

In [233]:
def get_last_name(name):    
    return name.split(' ', 1)[0]

In [234]:
def get_city(address):
    return 'Gunzgen'

def get_zip(address):
    return 4617

def get_address(address):
    if address:
        return address.replace(', Gunzgen', '')
    return None

In [235]:
for row_nr, row in enumerate(rows):
    cells = row
    if cells[0] and isinstance(cells[0], str) and cells[0].startswith('WA-'):
        # Subscription data
        period, subscriber_name, _, _, _, subscriber_number, *_ = rows[row_nr]
        _, invoice_address, _, _, mut_c, *_ = rows[row_nr + 1]
        _, _, _, _, _, start, _, _, _, _, _, building_address, *_ = rows[row_nr + 2]
        _, invoice_receiver, _, _, _, exit, _, _, _, _, _, building_category, *_ = rows[row_nr + 3]

        address = {
            'address': get_address(building_address),
            'zip': get_zip(building_address),
            'city': get_city(building_address),
            'country': 'CHE'
        }

        if address not in addresses:
            addresses.append(address)
            
        address_key = building_address
        building_key = subscriber_number
        
        building[subscriber_number] = {
            'address': address_key,   
            'building_category': building_category,
        }

        company = get_company(subscriber_name)

        invoice_address = address_data.get(subscriber_number, None)
        if not invoice_address:
            print("*problem", subscriber_number)
        
        subscriber[subscriber_number] = {
            'company': company,
            'last_name': None if company else get_last_name(subscriber_name),
            'alt_name': subscriber_name,
            'address': address_key,    
            'invoice_receiver': invoice_receiver,
            'invoice_address': invoice_address,
        }
        
    elif isinstance(cells[0], int):
        if cells[0] > 100:
            # Counter data
            (
                counter_nr, montage_nr, wohnungsbez, abl_code, _,
                montage_date, _, tarif, bez, tage, fkt, anz_zw, zw_alt_1, zw_alt_2,
                strg_z, add, zuge, zuga, folge
            ) = row
            counter[counter_nr] = {                
                'tarif': tarif,
                'bez': bez,
                'anz_zw': anz_zw,
                'zw_alt_1': zw_alt_1,
                'zw_alt_2': zw_alt_2,
                'strg_z': strg_z,
                'wohnungsbez': wohnungsbez,
                'building': building_key,
                'subscriber': subscriber_number
            }
            montage[counter_nr] = {                
                'date': montage_date,                
            }
        else:
            # Pricing data
            (tarif, bez, p_text, _ , _ , basis, anr, ansatz, betrag, tage,
             text, _ , zusatztext, _ , strgz, berz, strgg, berg, folge
            ) = row    

            # Product
            product_key = f"{tarif}_{anr}"
            
            description = '' if text or zusatztext else None
            if text:
                description = text
            if zusatztext:
                if description:
                    description += ', ' + zusatztext
                else:
                    description = zusatztext
            
            product[product_key] = {
                'tarif': tarif,                
                'bez': bez,
                'anr': anr,
                'name': p_text,
                'description': description,
                'price': ansatz,
                'strgz': strgz,
                'berz': berz,
            }

            # measurements
            if tarif in [14, 20]:
                measurements.append({
                    'counter': counter_nr,
                    'period': period,
                    'value': (zw_alt_1 if zw_alt_1 else 0) + 
                        (basis if basis else 0),
                    'consumption': basis
                })
                
            # Subscription
            if subscriber_number not in subscription:
                subscription[subscriber_number] = {
                    'subscriber': subscriber_number,
                    'building': subscriber_number,    
                    'products': [],
                    'start': start,
                    'exit': exit,        
                }
            subscription[subscriber_number]['products'].append(product_key)


In [236]:
for nr, data in product.items():
    print(data)

{'tarif': 5, 'bez': 'ZM', 'anr': 1, 'name': 'Zählermiete', 'description': None, 'price': 22.5, 'strgz': None, 'berz': None}
{'tarif': 14, 'bez': 'ARA', 'anr': 1, 'name': 'Abwasser ab 1.1.2020', 'description': None, 'price': 1.4, 'strgz': 'A', 'berz': 1}
{'tarif': 12, 'bez': 'GG W', 'anr': 1, 'name': 'Grundgebühr Wohnung', 'description': None, 'price': 60, 'strgz': None, 'berz': None}
{'tarif': 5, 'bez': 'ZM', 'anr': 2, 'name': 'Zählermiete', 'description': None, 'price': 35, 'strgz': None, 'berz': None}
{'tarif': 13, 'bez': 'GG In', 'anr': None, 'name': 'Grundgebühr Industrie', 'description': None, 'price': 375, 'strgz': None, 'berz': None}
{'tarif': 20, 'bez': 'ARA 1.75x', 'anr': 1, 'name': 'Abwasser 1.75x ab 01.01.2020', 'description': None, 'price': 2.45, 'strgz': 'A', 'berz': 1}
{'tarif': 6, 'bez': 'SpAn', 'anr': 1, 'name': 'Sprinkleranlage', 'description': None, 'price': 1953, 'strgz': None, 'berz': None}


In [237]:
measurements[-6]

{'counter': 18635347, 'period': 'WA-2402', 'value': 410, 'consumption': 42}

In [168]:
counter[22519056]

{'tarif': 1,
 'bez': 'Wa',
 'anz_zw': 1,
 'zw_alt_1': 39,
 'zw_alt_2': None,
 'strg_z': 'A',
 'wohnungsbez': None,
 'building': 417,
 'subscriber': 417}

In [169]:
montage[22519056]

{'date': '10.02.2022'}

subscription[22519056]

In [170]:
key = list(subscriber.keys())[-1]

In [171]:
subscriber[key]

{'company': 'Zweckverband ',
 'last_name': None,
 'alt_name': 'Zweckverband ',
 'address': 'Pumpwerk',
 'invoice_receiver': 'Zweckverband',
 'invoice_address': 'Klärstrasse 12'}

In [172]:
building[key]

{'address': 'Pumpwerk', 'building_category': None}

In [173]:
subscription[key]

{'subscriber': 417,
 'building': 417,
 'products': ['5_1', '14_1'],
 'start': '01.10.2008',
 'exit': None}

In [94]:
key

585

In [ ]:
subscriber